In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
raw_data_path = r'../../../../data/raw/freight_emission/'
processed_data_path = r'../../../../data/intermediate/test/freight_emission/'

In [3]:
filenames = list(filter(lambda x: '.XLSX' in x, os.listdir(raw_data_path)))
filenames

['EFA_ColdStart_Subsegm_LCV2025detailed.XLSX',
 'EFA_ColdStart_Subsegm_MC2025detailed.XLSX',
 'EFA_ColdStart_Vehcat_2025average.XLSX',
 'EFA_HOT_Subsegm_LCV2025detailed.XLSX',
 'EFA_HOT_Subsegm_MC2025detailed.XLSX',
 'EFA_HOT_Vehcat_2025average.XLSX']

# Check the consistency of EF

In [4]:
def check_and_fill_warm_emission_factors(df, vehicle_type='eBike', field_name='V_100%'):

    sub_df = df[df['Subsegment'] == vehicle_type]

    full_ef = df['Component'].unique()
    full_traffic_sit = df['TrafficSit'].unique()

    full_fields_list = df.columns.tolist()
    idx_V = full_fields_list.index(field_name)
    post_fields_list = full_fields_list[idx_V:]

    if sub_df.empty:
        raise ValueError(f"No records found for vehicle type '{vehicle_type}'")
    sample_eBike_record = sub_df.iloc[0]

    # create new records
    new_records = []
    for ef in full_ef:
        for ts in full_traffic_sit:
            if not ((sub_df['Component'] == ef) & (sub_df['TrafficSit'] == ts)).any():
                
                new_record = sample_eBike_record.to_dict()
                new_record['Component'] = ef
                new_record['TrafficSit'] = ts
                for field in post_fields_list:
                    new_record[field] = 0
                new_records.append(new_record)

    # Add new records to the dataframe
    if new_records:
        new_records_df = pd.DataFrame(new_records)
        df = pd.concat([df, new_records_df], ignore_index=True)

    return df


In [5]:
def check_and_fill_cold_emission_factors(df, vehicle_type='LCV petrol N1-III Euro-6c', field_name='%OfSubsegment'):

    sub_df = df[df['Subsegment'] == vehicle_type]

    full_ef = df['Component'].unique()
    full_traffic_sit = df['AmbientCondPattern'].unique()

    full_fields_list = df.columns.tolist()
    idx_V = full_fields_list.index(field_name)
    post_fields_list = full_fields_list[idx_V:]

    if sub_df.empty:
        raise ValueError(f"No records found for vehicle type '{vehicle_type}'")
    sample_eBike_record = sub_df.iloc[0]

    # create new records
    new_records = []
    for ef in full_ef:
        for ts in full_traffic_sit:
            if not ((sub_df['Component'] == ef) & (sub_df['AmbientCondPattern'] == ts)).any():
                
                new_record = sample_eBike_record.to_dict()
                new_record['Component'] = ef
                new_record['AmbientCondPattern'] = ts
                for field in post_fields_list:
                    new_record[field] = 0
                new_records.append(new_record)

    # Add new records to the dataframe
    if new_records:
        new_records_df = pd.DataFrame(new_records)
        df = pd.concat([df, new_records_df], ignore_index=True)

    return df

## Convert the sizeClasse of eBike into NA

In [6]:
def convert_eBike_sizeClass(row: pd.Series):
    if row['SizeClasse'] == 'not specified' and row['Subsegment'] == 'eBike':
        return 'NA' 
    else:
        return row['SizeClasse']       

# Replace the value of EFA for CO2e

In [7]:
def replace_efa4co2e(row: pd.Series):
    if row['Component'] == 'CO2e':
        return row['EFA_WTW']
    else:
        return row['EFA']

# Output

In [10]:
for filename in filenames:
    df = pd.read_excel(raw_data_path + filename)
    # Remove rows with error region/country info
    df = df.query("AmbientCondPattern != 'ØGermany'")
    if df.empty:
        pass
    else:
        print(f"Processing file {filename}")
        # Fill missing emission factors
        if 'Cold' in filename and 'MC' in filename:
            df = check_and_fill_cold_emission_factors(df, vehicle_type='eBike')
        elif 'Cold' in filename and 'LCV' in filename:
            df = check_and_fill_cold_emission_factors(df, vehicle_type='LCV petrol N1-III Euro-6c')
        elif 'HOT' in filename and 'MC' in filename:
            df = check_and_fill_warm_emission_factors(df, vehicle_type='eBike')
        elif 'HOT' in filename and 'LCV' in filename:
            df = check_and_fill_warm_emission_factors(df, vehicle_type='LCV petrol N1-III Euro-6c')
        else:
            # Average table
            pass
        
    if 'average' not in filename:
        # replace the value of 'EFA' with 'EFA_WTW' for emission factor - CO2e
        df['EFA'] = df.apply(lambda x: replace_efa4co2e(x), axis=1)

    if 'MC' in filename and not df.query("Subsegment == 'eBike'").empty:
        # Convert the SizeClasse for eBike
        df['SizeClasse'] = df.apply(lambda x: convert_eBike_sizeClass(x), axis=1)

    # check the consistency of the emission factors, and then fill the missing factors.
    df.to_csv(processed_data_path + filename[:-5] + "V2.csv.gz", sep=';', index=False, compression='gzip', encoding='utf-8-sig')
    print(f"File {filename} processed and saved to {processed_data_path + filename[:-5] + 'V2.csv.gz'}")


Processing file EFA_ColdStart_Subsegm_LCV2025detailed.XLSX
File EFA_ColdStart_Subsegm_LCV2025detailed.XLSX processed and saved to ../../../../data/intermediate/test/freight_emission/EFA_ColdStart_Subsegm_LCV2025detailedV2.csv.gz
File EFA_ColdStart_Subsegm_MC2025detailed.XLSX processed and saved to ../../../../data/intermediate/test/freight_emission/EFA_ColdStart_Subsegm_MC2025detailedV2.csv.gz
Processing file EFA_ColdStart_Vehcat_2025average.XLSX
File EFA_ColdStart_Vehcat_2025average.XLSX processed and saved to ../../../../data/intermediate/test/freight_emission/EFA_ColdStart_Vehcat_2025averageV2.csv.gz
Processing file EFA_HOT_Subsegm_LCV2025detailed.XLSX
File EFA_HOT_Subsegm_LCV2025detailed.XLSX processed and saved to ../../../../data/intermediate/test/freight_emission/EFA_HOT_Subsegm_LCV2025detailedV2.csv.gz
Processing file EFA_HOT_Subsegm_MC2025detailed.XLSX
File EFA_HOT_Subsegm_MC2025detailed.XLSX processed and saved to ../../../../data/intermediate/test/freight_emission/EFA_HOT_Su

# Test

In [8]:
test_data = pd.read_excel(raw_data_path + filenames[-2])
test_data

,Case,VehCat,Year,TrafficScenario,Component,RoadCat,TrafficSit,Gradient,IDSubsegment,Subsegment,...,EFA_weighted,EFA_weighted_0%,EFA_weighted_100%,AmbientCondPattern,EFA_WTT,EFA_WTT_0%,EFA_WTT_100%,EFA_WTW,EFA_WTW_0%,EFA_WTW_100%
0,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919200,MC 4S <=250cc preEuro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919210,MC 4S <=250cc Euro-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919220,MC 4S <=250cc Euro-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919230,MC 4S <=250cc Euro-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919240,MC 4S <=250cc Euro-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300025,MC2025detailed[4.2.2],motorcycle,2020,NaN,CO2e,NaN,URB/Access/50/St+Go2,0%,939650,MC 2S <=250cc Euro-5,...,NaN,NaN,NaN,NaN,19.542997,NaN,NaN,128.809875,NaN,NaN
300026,MC2025detailed[4.2.2],motorcycle,2020,NaN,CO2e,NaN,URB/Access/50/St+Go2,0%,939660,MC 2S <=250cc Euro-6,...,NaN,NaN,NaN,NaN,19.542997,NaN,NaN,128.809875,NaN,NaN
300027,MC2025detailed[4.2.2],motorcycle,2020,NaN,CO2e,NaN,URB/Access/50/St+Go2,0%,951201,eBike,...,NaN,NaN,NaN,NaN,0.567458,NaN,NaN,0.567458,NaN,NaN
300028,MC2025detailed[4.2.2],motorcycle,2020,NaN,CO2e,NaN,URB/Access/50/St+Go2,0%,951401,eScooter,...,NaN,NaN,NaN,NaN,11.269876,NaN,NaN,11.269876,NaN,NaN


In [9]:
test_data.query("AmbientCondPattern != 'ØGermany'")

,Case,VehCat,Year,TrafficScenario,Component,RoadCat,TrafficSit,Gradient,IDSubsegment,Subsegment,...,EFA_weighted,EFA_weighted_0%,EFA_weighted_100%,AmbientCondPattern,EFA_WTT,EFA_WTT_0%,EFA_WTT_100%,EFA_WTW,EFA_WTW_0%,EFA_WTW_100%
0,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919200,MC 4S <=250cc preEuro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919210,MC 4S <=250cc Euro-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919220,MC 4S <=250cc Euro-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919230,MC 4S <=250cc Euro-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MC2025detailed[4.2.2],motorcycle,2020,NaN,HC,NaN,RUR/MW/80/Freeflow,0%,919240,MC 4S <=250cc Euro-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300025,MC2025detailed[4.2.2],motorcycle,2020,NaN,CO2e,NaN,URB/Access/50/St+Go2,0%,939650,MC 2S <=250cc Euro-5,...,NaN,NaN,NaN,NaN,19.542997,NaN,NaN,128.809875,NaN,NaN
300026,MC2025detailed[4.2.2],motorcycle,2020,NaN,CO2e,NaN,URB/Access/50/St+Go2,0%,939660,MC 2S <=250cc Euro-6,...,NaN,NaN,NaN,NaN,19.542997,NaN,NaN,128.809875,NaN,NaN
300027,MC2025detailed[4.2.2],motorcycle,2020,NaN,CO2e,NaN,URB/Access/50/St+Go2,0%,951201,eBike,...,NaN,NaN,NaN,NaN,0.567458,NaN,NaN,0.567458,NaN,NaN
300028,MC2025detailed[4.2.2],motorcycle,2020,NaN,CO2e,NaN,URB/Access/50/St+Go2,0%,951401,eScooter,...,NaN,NaN,NaN,NaN,11.269876,NaN,NaN,11.269876,NaN,NaN


In [ ]:
test_data.query("Subsegment == 'LCV petrol N1-III Euro-6c' and TrafficSit == 'URB/Access/30/Heavy'")

In [ ]:
test_data_ = check_and_fill_warm_emission_factors(test_data, vehicle_type='LCV petrol N1-III Euro-6c')
test_data_.query("Subsegment == 'LCV petrol N1-III Euro-6c' and TrafficSit == 'URB/Access/30/Heavy'")

In [ ]:
test_data_.query("Subsegment == 'LCV petrol N1-III Euro-6c' and TrafficSit == 'URB/Access/30/Heavy'").iloc[-1, :]